# An Introduction to SageMaker Random Cut Forests

***ランダムカットフォレストアルゴリズムを用いた時系列データの教師なし学習による異常検知***

---

1. [Introduction](#Introduction)
1. [Setup](#Setup)
1. [Training](#Training)
1. [Inference](#Inference)
1. [Epilogue](#Epilogue)

## Introduction
***

Amazon SageMaker Random Cut Forest (RCF) は、データセット内の異常なデータポイントを検出するために設計されたアルゴリズムです。異常の検出が重要な場合の例としては、ウェブサイトのアクティビティが文字どおり急増した場合、温度データが周期的な動きから逸脱した場合、公共交通機関の利用者数の変化が特別なイベントの発生を反映している場合などが挙げられます。

このノートブックでは、SageMaker RCF アルゴリズムを使用して、Numenta Anomaly Benchmark (NAB) NYC Taxi データセット（6ヶ月間のニューヨーク市のタクシー利用者数を記録）で RCF モデルを学習させます。そして、このモデルを用いて、各データポイントに対して「異常スコア」を出すことで、異常な事象を予測します。このノートの主な目標は以下の通りです。

* Amazon SageMaker で使用するためのデータの取得、変換、保存の方法を学習します。
* データセットに対してAWS SageMakerのトレーニングジョブを作成し、RCFモデルを生成します。
* RCF モデルを使用して、Amazon SageMaker のエンドポイントで推論を実行します。

以下は、このノートブックで説明しない内容です。

* RCF モデルを深く理解する。
* Amazon SageMakerのRCFアルゴリズムがどのように動作するかを理解する。

より詳しく知りたい場合は、[SageMaker RCF Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/randomcutforest.html)をご覧ください。



## Setup

***



*このノートブックはPython 3 (Data Science)カーネルでml.t3.mediumインスタンスをAmazon SageMaker Studioで使った際に動作保証しています。*

最初のステップでは、AWSクレデンシャルをセットアップしてSageMakerが学習データと（学習の生成物である）モデルをクラウド上で扱えるようにします。



### Amazon S3 バケットの選択

最初に、オリジナルデータが格納されている場所と学習データとモデルを格納する場所を指定します。***このexample中で編集する必要があるのはこのセルだけです。*** 特に、下記のデータが必要です。


* `bucket` - このアカウントからアクセスできるS3バケット
* `prefix` - このノートブックからの入力データと出力データを格納するバケットの場所(デフォルトの値で十分です)
* `downloaded_data_bucket` - この[リンク](https://raw.githubusercontent.com/numenta/NAB/master/data/realKnownCause/nyc_taxi.csv) からダウンロードしたデータを格納するS3バケット(注：外部のバケット)
* `downloaded_data_prefix` - データが格納されるバケットの場所

In [ ]:
import boto3
import botocore
import sagemaker
import sys


bucket = (
    sagemaker.Session().default_bucket()
)  # Feel free to change to another bucket you have access to
prefix = "sagemaker/rcf-benchmarks"
execution_role = sagemaker.get_execution_role()
region = boto3.Session().region_name

# S3 bucket where the original data is downloaded and stored.
downloaded_data_bucket = f"sagemaker-sample-files"
downloaded_data_prefix = "datasets/tabular/anomaly_benchmark_taxi"


def check_bucket_permission(bucket):
    # check if the bucket exists
    permission = False
    try:
        boto3.Session().client("s3").head_bucket(Bucket=bucket)
    except botocore.exceptions.ParamValidationError as e:
        print(
            "Hey! You either forgot to specify your S3 bucket"
            " or you gave your bucket an invalid name!"
        )
    except botocore.exceptions.ClientError as e:
        if e.response["Error"]["Code"] == "403":
            print(f"Hey! You don't have permission to access the bucket, {bucket}.")
        elif e.response["Error"]["Code"] == "404":
            print(f"Hey! Your bucket, {bucket}, doesn't exist!")
        else:
            raise
    else:
        permission = True
    return permission


if check_bucket_permission(bucket):
    print(f"Training input/output will be stored in: s3://{bucket}/{prefix}")
if check_bucket_permission(downloaded_data_bucket):
    print(
        f"Downloaded training data will be read from s3://{downloaded_data_bucket}/{downloaded_data_prefix}"
    )

### サンプルデータの取得と分析

今回のデータは Numenta Anomaly Benchmark (NAB) NYC Taxi dataset [[1](https://github.com/numenta/NAB/blob/master/data/realKnownCause/nyc_taxi.csv)] から得たものです。データは[こちら](https://raw.githubusercontent.com/numenta/NAB/master/data/realKnownCause/nyc_taxi.csv)からダウンロードし、S3バケットに格納します。これらのデータは、6ヶ月間のニューヨーク市のタクシー利用者数を30分間隔で集約したものです。ニューヨークマラソン、感謝祭、クリスマス、元旦、雪嵐の日に異常なイベントが発生していることがあらかじめ分かっています。


> [1] https://github.com/numenta/NAB/blob/master/data/realKnownCause/nyc_taxi.csv

In [ ]:
%%time

import pandas as pd

data_filename = "NAB_nyc_taxi.csv"
s3 = boto3.client("s3")
s3.download_file(downloaded_data_bucket, f"{downloaded_data_prefix}/{data_filename}", data_filename)
taxi_data = pd.read_csv(data_filename, delimiter=",")

モデルを学習する前に、まずデータを調査することが重要です。もしかしたら、モデルに「ヒント」として提供できるような基本的なパターンや構造があるかもしれませんし、前処理で取り除けるようなノイズがあるかもしれません。生データは以下のようなものです。


In [ ]:
taxi_data.head()

データをプロットして見てみましょう。

In [ ]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt

matplotlib.rcParams["figure.dpi"] = 100

taxi_data.plot()

データを視覚化することで、パターンを認識することができます。例えば、データポイント6000番あたりで、いつもと違うことが起きています。また、タクシーの乗客数にも言えることですが、乗客の数に多少なりとも周期性があるように見受けられます。この異常値を調べるだけでなく、「正常な」データがどのようなものかを知るために、拡大してみましょう。



In [ ]:
taxi_data[5500:6500].plot()

ここで、タクシーの乗車数は、約50のデータポイントの長さの1つのモードを持つ、ほとんど周期的なものであることがわかります。実際、各データポイントは30分の乗車間隔で集計されており、このモードは長さ48です。また、1週間の長さである$336=48×7$のモードがもう一つあると予想されます。また、1日のうちでもっと小さな頻度も発生する。

例えば、上記のアノマリーを含む日中のデータは以下の通りです。


In [ ]:
taxi_data[5952:6000]

## 学習

***
次にタクシーデータにRandom Cut Forest (RCF)アルゴリズムを使って学習するためにSageMakerトレーニングジョブの設定をおこないます。

### ハイパーパラメータ

SageMaker RCFトレーニングジョブに特有のハイパーパラメータは以下のものです。

* **`num_samples_per_tree`** - 各木に送られるランダムにサンプルされたデータの数。一般的なルールとして、`1/num_samples_per_tree`がデータセット中の正常データに対する以上データの比になるように調整します。

* **`num_trees`** - 森の中に造られる木の数。それぞれの木は異なるサンプルデータから学習を行います。最終的なモデルはそれぞれの木で予測された予測スコアを平均して出力します。
* **`feature_dim`** - それぞれのデータポイントの次元数

RCFモデルのハイパーパラメータに加えて、学習が実行されるときのインスタンスタイプやデータが含まれるS3バケットやAWSアクセスロールを設定します。

In addition to these RCF model hyperparameters, we provide additional parameters defining things like the EC2 instance type on which training will run, the S3 bucket containing the data, and the AWS access role. Note that,

* 推奨するインスタンスタイプ: `ml.m4`, `ml.c4`, or `ml.c5`
* 現状の制限事項:
  * RCFアルゴリズムはGPUハードウェアをサポートしていません。

In [ ]:
from sagemaker import RandomCutForest

session = sagemaker.Session()

# specify general training job information
rcf = RandomCutForest(
    role=execution_role,
    instance_count=1,
    instance_type="ml.m4.xlarge",
    data_location=f"s3://{bucket}/{prefix}/",
    output_path=f"s3://{bucket}/{prefix}/output",
    num_samples_per_tree=512,
    num_trees=50,
)

# automatically upload the training data to S3 and run the training job
rcf.fit(rcf.record_set(taxi_data.value.to_numpy().reshape(-1, 1)))


> `===== Job Complete =====`

の文字が出力ログの下部に表示された場合、学習が成功してRCFモデルが指定された出力パスにアップロードされています。AWS SageMakerコンソールからも学習の進捗や情報を確認することができます。下のセルを実行して出てきた学習ジョブ名をコンソールの学習ジョブから見つけクリックしてみましょう。


In [ ]:
print(f"Training job name: {rcf.latest_training_job.job_name}")

## 推論

***

学習したRandom Cut Forestモデル自体を持っていても何も起こりません。データを使って推論し、異常スコアを計算してみましょう。このexampleでは学習に使ったと同じ入力時系列データの異常スコアを計算します。

ここでは、上で行った学習ジョブ（の変数）からSageMaker Python SDKの`deploy()`関数を使って推論エンドポイントを作成します。作成するエンドポイントのインスタンスタイプとインスタンスの初期台数を指定します。推論速度とコストの効率がよい`ml.c5`インスタンスをおすすめします。デプロイの完了まで数分かかります。



In [ ]:
rcf_inference = rcf.deploy(initial_instance_count=1, instance_type="ml.m4.xlarge")

おめでとうございます！これでSageMaker RCF推論エンドポイントができあがりました。下記のセルを実行してエンドポイント名を確認したのち、SageMakerのコンソール上から"エンドポイント"タブから該当するエンドポイントをクリックすることでエンドポイントの設定とステータスを確認することができます。

In [ ]:
print(f"Endpoint name: {rcf_inference.endpoint}")

### Data Serialization/Deserialization

作成したエンドポイントにはさまざまなフォーマットのデータを使うことができます。このexampleでは、CSVフォーマットのデータを使います。他に対応可能なフォーマットはJSONとRecordIOです。SageMaker Python SDKユーティリティーの`csv_serializer`と`json_deserializer`を使ってエンドポイントの設定を行います。



In [ ]:
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import JSONDeserializer

rcf_inference.serializer = CSVSerializer()
rcf_inference.deserializer = JSONDeserializer()

CSVフォーマットのデータを推論エンドポイントに通して、結果をプロットしてみましょう。このノートブック上で取り扱っているデータはNumpy NDArray形式ですが、上でシリアライザーを設定するとデータをNDArrayから自動的に変換してくれます。

まずは、最初の6個のデータポイントを推論して結果を確認してみましょう。

In [ ]:
taxi_data_numpy = taxi_data.value.to_numpy().reshape(-1, 1)
print(taxi_data_numpy[:6])
results = rcf_inference.predict(
    taxi_data_numpy[:6], initial_args={"ContentType": "text/csv", "Accept": "application/json"}
)

### 異常スコアの計算

それでは、完全なタクシーデータセットの異常スコアを計算してプロットします。

In [ ]:
results = rcf_inference.predict(taxi_data_numpy)
scores = [datum["score"] for datum in results["scores"]]

# add scores to taxi data frame and print first few values
taxi_data["score"] = pd.Series(scores, index=taxi_data.index)
taxi_data.head()

In [ ]:
fig, ax1 = plt.subplots()
ax2 = ax1.twinx()

#
# *Try this out* - change `start` and `end` to zoom in on the
# anomaly found earlier in this notebook
#
start, end = 0, len(taxi_data)
# start, end = 5500, 6500
taxi_data_subset = taxi_data[start:end]

ax1.plot(taxi_data_subset["value"], color="C0", alpha=0.8)
ax2.plot(taxi_data_subset["score"], color="C1")

ax1.grid(which="major", axis="both")

ax1.set_ylabel("Taxi Ridership", color="C0")
ax2.set_ylabel("Anomaly Score", color="C1")

ax1.tick_params("y", colors="C0")
ax2.tick_params("y", colors="C1")

ax1.set_ylim(0, 40000)
ax2.set_ylim(min(scores), 1.4 * max(scores))
fig.set_figwidth(10)

以下では、平均点から3標準偏差（約99.9パーセンタイル）以上のスコアを持つデータポイントをプリントし、プロットしています。


In [ ]:
score_mean = taxi_data["score"].mean()
score_std = taxi_data["score"].std()
score_cutoff = score_mean + 3 * score_std

anomalies = taxi_data_subset[taxi_data_subset["score"] > score_cutoff]
anomalies


以下は、この期間にニューヨークで発生した既知のイベントの一覧です。

* `2014-11-02` - NYC Marathon
* `2015-01-01` - New Year's Eve
* `2015-01-27` - Snowstorm

このアルゴリズムは、これらのイベントと他の多くのイベントを捉えることができたことがわかりました。以下では、これらの異常値をスコアプロットに追加しています。


In [ ]:
ax2.plot(anomalies.index, anomalies.score, "ko")
fig

現在のハイパーパラメータの選択では、3標準偏差の閾値は、乗車率プロットに見られる異常を捉えることができるものの、細かい摂動や異常な挙動に対してかなり敏感であることがわかります。SageMakerのRCFモデルに木を追加すれば、より大きなデータセットを使用するのと同様に、結果を滑らかにすることができます。


### エンドポイントの停止と削除

最後に、ノートブックを閉じる前にエンドポイントを削除します。

これを行うには、以下のセルを実行します。あるいは、SageMaker コンソールの「エンドポイント」タブに移動し、変数 `endpoint_name` に格納されている名前のエンドポイントを選択し、「アクション」ドロップダウン・メニューから「削除」を選択することも可能です。



In [ ]:
sagemaker.Session().delete_endpoint(rcf_inference.endpoint)

## エピローグ

---

Amazon SageMakerのランダムカットフォレストを用いて、タクシー利用者のデータセットから異常なデータポイントを検出しました。これらのデータでは、異常は乗車率が異常に高いか低いときに発生しました。データを変換することで、RCFアルゴリズムは、例えば、データが周期性を破ったときや、グローバルな挙動が特徴的に変化したときにも検出することができます。

アルゴリズムの性能を向上させるには、データの種類によっていくつかの方法があります。例えば、適切なトレーニングセットを使用する方法です。あるデータセットが「正常な」動作の特徴であることが分かっている場合、そのデータセットでトレーニングを行い、正常時の振る舞いを学習させることで、より正確に「異常な」データを特徴付けることができます。

もう一つの改良点は、"shingling "と呼ばれるデータのウィンドウ（窓）処理技術を利用することです。これは、上記のニューヨークのタクシーデータセットのような、周期的なデータを扱うときに特に有効である。このアイデアは、$P$個のデータポイントの周期を、特徴量（次元）の長さが$P$の単一のデータポイントとして扱い、これらの特徴ベクトルに対してRCFアルゴリズムを実行することです。つまり、元のデータが点$x_1, x_2, \dots, x_N \in \mathbb{R}$で構成されている場合、以下のような変換を実行します。




```
data = [[x_1],            shingled_data = [[x_1, x_2, ..., x_{P}],
        [x_2],    --->                     [x_2, x_3, ..., x_{P+1}],
        ...                                ...
        [x_N]]                             [x_{N-P}, ..., x_{N}]]

```

In [ ]:
import numpy as np


def shingle(data, shingle_size):
    num_data = len(data)
    shingled_data = np.zeros((num_data - shingle_size, shingle_size))

    for n in range(num_data - shingle_size):
        shingled_data[n] = data[n : (n + shingle_size)]
    return shingled_data


# single data with shingle size=48 (one day)
shingle_size = 48
prefix_shingled = "sagemaker/randomcutforest_shingled"
taxi_data_shingled = shingle(taxi_data.values[:, 1], shingle_size)
print(taxi_data_shingled)

新しいトレーニングジョブと推論エンドポイントを作成します。(上記で作成したエンドポイントは1次元データで学習しているため、再利用できないことに注意してください)。

In [ ]:
session = sagemaker.Session()

# specify general training job information
rcf = RandomCutForest(
    role=execution_role,
    instance_count=1,
    instance_type="ml.m4.xlarge",
    data_location=f"s3://{bucket}/{prefix_shingled}/",
    output_path=f"s3://{bucket}/{prefix_shingled}/output",
    num_samples_per_tree=512,
    num_trees=50,
)

# automatically upload the training data to S3 and run the training job
rcf.fit(rcf.record_set(taxi_data_shingled))

In [ ]:
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import JSONDeserializer

rcf_inference = rcf.deploy(initial_instance_count=1, instance_type="ml.m4.xlarge")

rcf_inference.serializer = CSVSerializer()
rcf_inference.deserializer = JSONDeserializer()

In [ ]:
print(rcf_inference.content_type, rcf_inference.accept)

上記の推論エンドポイントを用いて、shinglingをおこなったデータに関連する異常スコアを計算します。

In [ ]:
# Score the shingled datapoints
results = rcf_inference.predict(
    taxi_data_shingled, initial_args={"ContentType": "text/csv", "Accept": "application/json"}
)
scores = np.array([datum["score"] for datum in results["scores"]])

# compute the shingled score distribution and cutoff and determine anomalous scores
score_mean = scores.mean()
score_std = scores.std()
score_cutoff = score_mean + 3 * score_std

anomalies = scores[scores > score_cutoff]
anomaly_indices = np.arange(len(scores))[scores > score_cutoff]

print(anomalies)

最後に、shinglingをおこなったデータのスコアを元のデータセットの上にプロットし、異常スコア閾値を超えるスコアをマークします。

In [ ]:
fig, ax1 = plt.subplots()
ax2 = ax1.twinx()

#
# *Try this out* - change `start` and `end` to zoom in on the
# anomaly found earlier in this notebook
#
start, end = 0, len(taxi_data)
taxi_data_subset = taxi_data[start:end]

ax1.plot(taxi_data["value"], color="C0", alpha=0.8)
ax2.plot(scores, color="C1")
ax2.scatter(anomaly_indices, anomalies, color="k")

ax1.grid(which="major", axis="both")
ax1.set_ylabel("Taxi Ridership", color="C0")
ax2.set_ylabel("Anomaly Score", color="C1")
ax1.tick_params("y", colors="C0")
ax2.tick_params("y", colors="C1")
ax1.set_ylim(0, 40000)
ax2.set_ylim(min(scores), 1.4 * max(scores))
fig.set_figwidth(10)

この特定のshinglingのサイズ、ハイパーパラメータの選択、および異常カットオフ閾値の設定で、主要な異常事象をより明確に捕らえることができることがわかります（t=6000付近のスパイク、t=9000およびt=10000付近のディップ）。一般的に、木の数、サンプルサイズ、異常スコアのカットオフ値は、データサイエンティストが望ましい結果を得るために実験的に求める必要があります。ラベル付きテストデータセットを利用できる場合は、異常検出アルゴリズムの一般的な精度メトリクスを得ることができます。Amazon SageMaker Random Cut Forestの詳細については、以下を参照してください。

[AWS Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/randomcutforest.html).

最後にエンドポイントを忘れずに削除します。

In [ ]:
sagemaker.Session().delete_endpoint(rcf_inference.endpoint)